# Parte 1: DCClasficador de sonidos 

Nota de los ayudantes: Por motivos de desempeño de las redes neuronales, recomendamos realizar esta tarea en Google Colab.

Las siguientes tres celdas te permiten descargar el dataset directamente desde Kaggle (una plataforma de inteligencia artificial y ciencia de datos). Esto puede facilitar la descarga, pero siéntete libre de utilizar otros métodos para acceder al set de datos.

In [ ]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [ ]:
# necesitas autenticarte para descargar el dataset directamente desde kaggle
# más información en https://www.kaggle.com/docs/api

od.download(
    "https://www.kaggle.com/datasets/chrisfilo/urbansound8k"
)

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import librosa
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from datetime import datetime

In [ ]:
# Creamos una semilla para que los resultados sean replicables
n_alumno = # tu número de alumno
np.random.seed(n_alumno)

In [ ]:
# Definimos el path al csv
path =  # aquí va la ruta donde almacenaste la carpeta con datos

# Importamos el csv
meta_data = pd.read_csv(path + "UrbanSound8K.csv")

# Veamos las primeras 5 observaciones
meta_data.head()

In [ ]:
# Usamos una función que convierte el audio en mfcc
def features_extract(file):

    # Cargamos el audio
    sound_wave, sample_rate = librosa.load(path = file, res_type= 'kaiser_fast')

    # Obtenemos sus mfcc's
    feature = librosa.feature.mfcc(y = sound_wave, sr = sample_rate, n_mfcc = 40)

    # Retornamos los mfcc's del audio
    return feature

# Hacemos una lista vacía para guardar los mfcc de las canciones
extracted = []

# Hacemos otra para los features no escalados
extracted_cnn = []

# Iteramos sobre cada fila del csv
for index_num, row in tqdm(meta_data.iterrows()):

    # Obtenemos el path del audio
    file_name = os.path.join(os.path.abspath(path), 'fold' + str(row["fold"]) + '/', str(row['slice_file_name'])) 

    # Obtenemos la clase
    final_class_labels = row['class']  

    # Usamos la función definida arriba para calcular sus mfcc's
    data = features_extract(file_name)

    # Calculamos la media de cada fila de los mfcc
    scaled_feature = np.mean(data.T, axis = 0)    

    # Guardamos el audio pre-procesado en mfcc's y su clase en una lista 
    extracted.append([scaled_feature, final_class_labels])

    # Cambiamos las dimensiones del audio para la parte de CNN
    X_ = cv2.resize(data, (40, 174), interpolation = cv2.INTER_LINEAR)

    # Guardamos el audio cambiado de tamaño en una lista
    extracted_cnn.append(X_)

# Creamos un dataframe con los predictores y la variable respuesta
datos = pd.DataFrame(extracted, columns = ["features", "response"])

# Definimos los predictores
X = np.array(datos["features"].tolist())

# Definimos la variable respuesta
y = np.array(datos["response"].tolist())

# Definimos el x para la red neuronal convolucional
X_cnn = np.array(extracted_cnn)[..., np.newaxis]

# Lista con las clases en orden
clases = ["Air Conditioner", "Car Horn", "Children Playing", "Dog Bark", "Drilling", "Engine Idling", "Gun Shot", "Jackhammer", "Siren", "Street Music"]

## Actividad 1: Comprendientdo los datos (0.2 pts.)


- ¿Qué son y para qué sirven los MFCC's?


- Si tuvieramos pocos audios para entrenar un modelo ¿Qué técnicas porías usar para enriquecer el set de datos?


## Actividad 2: Estructura de una red neuronal densa (0.5 pts.)

- Explica, en términos generales, en qué consiste una red neuronal densa

- Explica qué es una función de activación, y comenta sobre qué propiedades debe cumplir dicha función para poder ser utilizada en una red neuronal. Menciona un ejemplo en particular de función de activación, y fundamenta por qué podría ser utilizada para construir una red neuronal.

- Investiga sobre el problema de _vanishing gradient_ en el entreamiento de redes neuronales, y cómo la función de activación ReLU proporciona ventajas sobre otras funciones de activación ante este problema.

- Investiga en qué consiste una capa _softmax_, especificando qué hace y cuál su utilidad en los problemas de clasificación entre varias categorías. 

- Investiga sobre el optimizador Adam. Explica cómo funciona y qué lo diferencia de SGD (Stochastic Gradient Descent) ¿Qué beneficios tiene y por qué podría ser de utilidad en este contexto?.

## Actividad 3: Red Neuronal Densa (0.8 pts.)


- Implementa una red neuronal densa multicapa. Esta debe ser capaz de recibir el set de datos con los MFCC's y entregar un output de 10 dimensiones.

In [ ]:
# Separamos en entrenamiento, testeo y validación.
# Definimos las proporciones de cada base

train_prop = # proporción de entrenamiento
test_prop = # proporción de testeo
val_prop = # proporción de validación

# Separamos nuestras variables en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = test_prop, 
    random_state = n_alumno
    )

# Ahora con la de entrenamiento hacemos la de validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, 
    y_train, 
    test_size = val_prop / (train_prop + test_prop), 
    random_state = n_alumno
    )

- Grafica cómo varía el accuracy y la función de pérdida con las épocas para los conjuntos de entrenamiento y validación, e interpreta lo que ves.


- Evalua el accuracy en el set de test. Comenta los resultados brevemente.

## Actividad 4: Introducción a las Redes Neuronales Convolucionales (0.3 pts.)

- Lee el siguiente artículo: https://www.ibm.com/cloud/learn/convolutional-neural-networks?utm_medium=OSocial&utm_source=Youtube&utm_content=WAIWW&utm_id=YTDescription-101-What-are-CNNs-LH-CNNs-Guide y explica brevemente en qué consisten las redes neuronales convolucionales (CNN). En tu respuesta, explica la función de los siguientes componenetes de una CNN:
  - Convolutional layer
  - Pooling layer
  - Fully connected layer 

- ¿Cuales ventajas tiene una CNN con respecto a las redes neuronales convencionales al trabajar con audio?

## Actividad 5: Creando una CNN clasificadora (0.8 pts.)

Implementa una red neuronal convolucional para predecir las categorías de los sonidos disponibles. Grafica una matriz de confusión para las categorías predichas.

In [ ]:
# Separamos en entrenamiento, testeo y validación.

# Definimos las proporciones de cada base para la cnn
train_prop = # proporción entrenamiento
test_prop = # proporción testeo
val_prop = # proporción validación

# Separamos nuestras variables en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(
    X_cnn, 
    y, 
    test_size = test_prop, 
    random_state = n_alumno
    )

# Ahora con la de entrenamiento hacemos la de validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, 
    y_train, 
    test_size = val_prop / (train_prop + test_prop), 
    random_state = n_alumno
    )

# Se definen las dimensiones de entrada a la red neuronal convolucional
input_shape_cnn = (X_train_cnn.shape[1], X_train_cnn.shape[2], X_train_cnn.shape[3])

## Actividad 6: Comparación de modelos (0.2 pts.)




- ¿Cuales son las clases que más se confunden en cada una de las redes neuronales? ¿Por qué crees que ocurre?

- ¿Cuáles clases son las que peor predicen los modelos? ¿A qué podría deberse esto?

## Actividad 7: Probando la CNN (0.2 pts.)

- Mostrar el nombre de la clase predicha para este audio y mostrar el audio

- Si la clase predicha no coincide en el sonido del audio, explicar a qué podría deberse ¿Hay ruido de fondo? ¿El sonido se parece a otra clase?

In [ ]:
# Función mencionada en el enunciado que recibe el path a cierto audio .wav
def transform(path):

    # Utiliza la función entregada más arriba para obtener los MFCC's del audio
    audio = features_extract(path)

    # Convierte a dimensiones apropiadas
    X_ = cv2.resize(audio, (40, 174), interpolation = cv2.INTER_LINEAR)

    # Se agrega una dimensión
    X = X_[..., np.newaxis].reshape(1, 174, 40, 1)

    # Retorna el audio transformado, listo para que el modelo prediga su clase
    return X